In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#x_train = np.load('/content/drive/My Drive/dlhw1p2/train.npy', allow_pickle=True)

Mounted at /content/drive


In [2]:
#! cp "/content/drive/My Drive/Colab Notebooks/fb_mmf/default_stvqa_testvqa_val_predictions/evaluation_metrics.py" .
#import evaluation_metrics

In [3]:
import json
import requests
import re
from collections import Counter

import os
import numpy as np
from PIL import Image

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import time

from PIL import Image
import requests
from io import BytesIO

import torchvision.models as models
import torchvision.transforms as transforms

from torch.autograd import Variable

# Install required libraries
#!pip install torch>=1.2.0 tensorboard future tqdm
#from torch.utils.tensorboard import SummaryWriter
#from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [4]:
!cd "/content/drive/My Drive/Multimodal ML (11-777)/textvqa_data"

In [5]:
url = 'https://dl.fbaipublicfiles.com/textvqa/data/TextVQA_0.5.1_val.json'
r = requests.get(url)

train = r.json()
#print(r.json())

In [6]:
len(train['data'])

5000

In [7]:
train['data'][10]

{'answers': ['off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off'],
 'flickr_300k_url': 'https://c2.staticflickr.com/3/2686/4333070249_384da2502d_z.jpg',
 'flickr_original_url': 'https://c1.staticflickr.com/3/2686/4333070249_6b317f3a62_o.jpg',
 'image_classes': ['Food', 'Light switch'],
 'image_height': 683,
 'image_id': '4dc59c7e57fa8968',
 'image_width': 1024,
 'question': 'are these switches on or off?',
 'question_id': 34612,
 'question_tokens': ['are', 'these', 'switches', 'on', 'or', 'off'],
 'set_name': 'val'}

In [8]:
training_df = pd.DataFrame(train['data'])

In [9]:
training_df.head()

,question,image_id,image_classes,flickr_original_url,flickr_300k_url,image_width,image_height,answers,question_tokens,question_id,set_name
0,what is the brand of this camera?,003a8ae2ef43b901,"[Cassette deck, Printer, Medical equipment, Co...",https://farm2.staticflickr.com/4/5566811_bc00d...,https://farm2.staticflickr.com/4/5566811_bc00d...,1024,664,"[nous les gosses, dakota, clos culombu, dakota...","[what, is, the, brand, of, this, camera]",34602,val
1,what does the small white text spell?,b9dc400eb20bad64,[Billboard],https://farm5.staticflickr.com/4136/4920614800...,https://c7.staticflickr.com/5/4136/4920614800_...,1024,683,"[copenhagen, copenhagen, copenhagen, copenhage...","[what, does, the, small, white, text, spell]",34603,val
2,what kind of beer is this?,2b538a43dd933fc1,"[Surfboard, Drink, Bottle, Wine]",https://farm2.staticflickr.com/3519/5721283932...,https://c8.staticflickr.com/4/3519/5721283932_...,1024,1024,"[ale, sublimely self-righteous ale, stone, ale...","[what, kind, of, beer, is, this]",34604,val
3,what brand liquor is on the right?,831bcec304a17054,"[Beer, Drink, Bottle, Wine]",https://farm5.staticflickr.com/5590/1522389734...,https://c6.staticflickr.com/6/5590/15223897342...,1024,576,"[bowmore , bowmore, bowmore, bowmore, bowmore,...","[what, brand, liquor, is, on, the, right]",34605,val
4,how long has the drink on the right been aged?,831bcec304a17054,"[Beer, Drink, Bottle, Wine]",https://farm5.staticflickr.com/5590/1522389734...,https://c6.staticflickr.com/6/5590/15223897342...,1024,576,"[10 years, 10 year, 10 years, 10 years , 10 ye...","[how, long, has, the, drink, on, the, right, b...",34606,val


In [10]:
len(training_df)

5000

In [11]:
predictions_df = pd.read_json("/content/drive/My Drive/Colab Notebooks/fb_mmf/default_stvqa_testvqa_val_predictions/textvqa_run_val.json")

In [12]:
predictions_df.head()

,question_id,image_id,answer,pred_source
0,34702,9dbf0225475ea1ae,0183995551,[OCR]
1,39531,bb807f6c33b9f04c,5,[VOCAB]
2,35202,89d9552a610e6a67,8,[VOCAB]
3,38253,bb14eb04b368614b,laura,[OCR]
4,35028,17f184cad1ba4d19,70g,[OCR]


In [13]:
len(set(predictions_df['question_id'])) #length of validation dataset

5000

In [14]:
len(set(training_df['question_id']))

5000

In [15]:
#make sure all our predictions can be checked with the answers in the training set (we want this length to be 0)
set(predictions_df['question_id']).difference(set(training_df['question_id']))

set()

In [16]:
x, y = [], []
for i in range(5):
  pred_values = predictions_df.iloc[i]
  question_id = pred_values['question_id']
  actual_values = training_df[training_df['question_id'] == question_id]
  print(pred_values['answer'])
  x.append(pred_values['answer'])
  print(actual_values['answers'].item())
  y.append(actual_values['answers'].item())

0183995551
['018 399 5551', '018 399 5551', '0183995551', '018-319-5551', '0183995551', '018 399 5551', '018 399 5551', '0183000200', '018 399 5551', '018 399 5551']
5
['5', '5', '5', '5', '5', '60', '5', '5', '5', '5']
8
['38', '36', '38', '30', '36', '36', '38', '36', '38', '36']
laura
['they', 'the family, including laura', 'they all', 'they all laughed', 'they all laughed', 'they all did', 'they', 'they all laughed', 'all of them', 'they']
70g
['3.50', '3.50', '3.50', '70g', '3.50', '3.50', '3.50', '3.50', '3.50', '3.50']


TextVQA Evaluation Metric

Code: 

https://github.com/GT-Vision-Lab/VQA/blob/master/PythonEvaluationTools/vqaEvaluation/vqaEval.py

In [17]:
manualMap = {'none': '0', 'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9', 'ten': '10'}
articles = ['a', 'an', 'the']
periodStrip  = re.compile("(?!<=\d)(\.)(?!\d)")
commaStrip   = re.compile("(\d)(\,)(\d)")
punct = [';', r"/", '[', ']', '"', '{', '}', '(', ')', '=', '+', '\\', '_', '-', '>', '<', '@', '`', ',', '?', '!']
contractions = {"aint": "ain't", "arent": "aren't", "cant": "can't", "couldve": "could've", "couldnt": "couldn't", \
		"couldn'tve": "couldn't've", "couldnt've": "couldn't've", "didnt": "didn't", "doesnt": "doesn't", "dont": "don't", "hadnt": "hadn't", \
		"hadnt've": "hadn't've", "hadn'tve": "hadn't've", "hasnt": "hasn't", "havent": "haven't", "hed": "he'd", "hed've": "he'd've", \
		"he'dve": "he'd've", "hes": "he's", "howd": "how'd", "howll": "how'll", "hows": "how's", "Id've": "I'd've", "I'dve": "I'd've", \
		"Im": "I'm", "Ive": "I've", "isnt": "isn't", "itd": "it'd", "itd've": "it'd've", "it'dve": "it'd've", "itll": "it'll", "let's": "let's", \
		"maam": "ma'am", "mightnt": "mightn't", "mightnt've": "mightn't've", "mightn'tve": "mightn't've", "mightve": "might've", \
		"mustnt": "mustn't", "mustve": "must've", "neednt": "needn't", "notve": "not've", "oclock": "o'clock", "oughtnt": "oughtn't", \
		"ow's'at": "'ow's'at", "'ows'at": "'ow's'at", "'ow'sat": "'ow's'at", "shant": "shan't", "shed've": "she'd've", "she'dve": "she'd've", \
		"she's": "she's", "shouldve": "should've", "shouldnt": "shouldn't", "shouldnt've": "shouldn't've", "shouldn'tve": "shouldn't've", \
		"somebody'd": "somebodyd", "somebodyd've": "somebody'd've", "somebody'dve": "somebody'd've", "somebodyll": "somebody'll", \
		"somebodys": "somebody's", "someoned": "someone'd", "someoned've": "someone'd've", "someone'dve": "someone'd've", \
		"someonell": "someone'll", "someones": "someone's", "somethingd": "something'd", "somethingd've": "something'd've", \
		"something'dve": "something'd've", "somethingll": "something'll", "thats": "that's", "thered": "there'd", "thered've": "there'd've", \
		"there'dve": "there'd've", "therere": "there're", "theres": "there's", "theyd": "they'd", "theyd've": "they'd've", \
		"they'dve": "they'd've", "theyll": "they'll", "theyre": "they're", "theyve": "they've", "twas": "'twas", "wasnt": "wasn't", \
		"wed've": "we'd've", "we'dve": "we'd've", "weve": "we've", "werent": "weren't", "whatll": "what'll", "whatre": "what're", \
		"whats": "what's", "whatve": "what've", "whens": "when's", "whered": "where'd", "wheres": "where's", "whereve": "where've", \
		"whod": "who'd", "whod've": "who'd've", "who'dve": "who'd've", "wholl": "who'll", "whos": "who's", "whove": "who've", "whyll": "why'll", \
		"whyre": "why're", "whys": "why's", "wont": "won't", "wouldve": "would've", "wouldnt": "wouldn't", "wouldnt've": "wouldn't've", \
		"wouldn'tve": "wouldn't've", "yall": "y'all", "yall'll": "y'all'll", "y'allll": "y'all'll", "yall'd've": "y'all'd've", \
		"y'alld've": "y'all'd've", "y'all'dve": "y'all'd've", "youd": "you'd", "youd've": "you'd've", "you'dve": "you'd've", \
		"youll": "you'll", "youre": "you're", "youve": "you've"}

In [18]:
def evaluate(labels, preds):
  assert(len(labels) == len(preds))
  accuracies = []
  for resAns in preds:
    resAns = resAns.replace('\n', ' ')
    resAns = resAns.replace('\t', ' ')
    resAns = resAns.strip()
    resAns = processPunctuation(resAns)
    resAns = processDigitArticle(resAns)

  process_labels = []
  for list_answers in labels:
    process_labels.append([processPunctuation(item) for item in list_answers])

  for i in range(len(process_labels)):
    list_answers = labels[i]
    incorrect = [item for item in list_answers if item!=preds[i]]
    correct = [item for item in list_answers if item==preds[i]]
    acc = min(1, float(len(correct))/3)
    accuracies.append(acc)
	
  avg_acc = float(sum(accuracies))/len(accuracies)
  assert(len(accuracies) == len(preds) == len(process_labels))
  return(accuracies, avg_acc)

In [19]:
def processPunctuation(inText):
	outText = inText
	for p in punct:
		if (p + ' ' in inText or ' ' + p in inText) or (re.search(commaStrip, inText) != None):
			outText = outText.replace(p, '')
		else:
			outText = outText.replace(p, ' ')	
			outText = periodStrip.sub("",
				outText,
				re.UNICODE)
	return outText

In [20]:
def processDigitArticle(inText):
	outText = []
	tempText = inText.lower().split()
	for word in tempText:
		word = manualMap.setdefault(word, word)
		if word not in articles:
			outText.append(word)
		else: pass
    
	for wordId, word in enumerate(outText):
		if word in contractions: 
			outText[wordId] = contractions[word]
			outText = ' '.join(outText)
	return outText

In [21]:
evaluate(y, x)

([0.6666666666666666, 1, 0.0, 0.0, 0.3333333333333333], 0.39999999999999997)

In [22]:
combo_df = pd.DataFrame()
#range of len(predictions_df)
for i in range(len(predictions_df)):
  pred_values = predictions_df.iloc[i]
  question_id = pred_values['question_id']
  temp_df = training_df[training_df['question_id'] == question_id].set_index('question_id')
  #print(len(temp_df))
  #print(pred_values)
  #print(temp_df)
  temp_df['answer'] = pred_values['answer']
  temp_df['pred_source'] = [pred_values['pred_source']] #list
  combo_df = pd.concat([combo_df, temp_df], axis=0)

In [23]:
combo_df

,question,image_id,image_classes,flickr_original_url,flickr_300k_url,image_width,image_height,answers,question_tokens,set_name,answer,pred_source
question_id,,,,,,,,,,,,
34702,what is the first phone number listed on the b...,9dbf0225475ea1ae,[Billboard],https://c7.staticflickr.com/7/6090/6136207758_...,https://c2.staticflickr.com/7/6090/6136207758_...,1024,768,"[018 399 5551, 018 399 5551, 0183995551, 018-3...","[what, is, the, first, phone, number, listed, ...",val,0183995551,[OCR]
39531,what is this players number?,bb807f6c33b9f04c,"[Shorts, Boy, Person, Volleyball, Woman, Man, ...",https://c5.staticflickr.com/5/4082/5440798267_...,https://c1.staticflickr.com/5/4082/5440798267_...,1024,768,"[5, 5, 5, 5, 5, 60, 5, 5, 5, 5]","[what, is, this, players, number]",val,5,[VOCAB]
35202,what number does the umpire have on?,89d9552a610e6a67,"[Boy, Person, Baseball glove, Man, Sports equi...",https://farm8.staticflickr.com/6086/6054663369...,https://c2.staticflickr.com/7/6086/6054663369_...,1024,768,"[38, 36, 38, 30, 36, 36, 38, 36, 38, 36]","[what, number, does, the, umpire, have, on]",val,8,[VOCAB]
38253,who all laughed?,bb14eb04b368614b,"[Person, Book]",https://c4.staticflickr.com/1/304/19620148836_...,https://c2.staticflickr.com/1/304/19620148836_...,1024,1024,"[they, the family, including laura, they all, ...","[who, all, laughed]",val,laura,[OCR]
35028,how much is the post card chocolate?,17f184cad1ba4d19,"[Building, Book]",https://c2.staticflickr.com/4/3811/11014312304...,https://c3.staticflickr.com/4/3811/11014312304...,1024,768,"[3.50, 3.50, 3.50, 70g, 3.50, 3.50, 3.50, 3.50...","[how, much, is, the, post, card, chocolate]",val,70g,[OCR]
...,...,...,...,...,...,...,...,...,...,...,...,...
37476,what state is the license plate for?,04bc55abd830f147,"[Vehicle, Car, Surfboard, Vehicle registration...",https://farm8.staticflickr.com/2175/2345080536...,https://c4.staticflickr.com/3/2175/2345080536_...,1024,768,"[california, california, california, santa mon...","[what, state, is, the, license, plate, for]",val,california,[VOCAB]
35626,what number is on the back of the man in red ...,015efa517e260a6c,"[Football, Ball, Boy, Person, Man, Plant, Spor...",https://c5.staticflickr.com/3/2424/3697384670_...,https://c1.staticflickr.com/3/2424/3697384670_...,1024,617,"[7, 7, 24, 7, 14, 2, 7, 7, unanswerable, 7]","[what, number, is, on, the, back, of, the, man...",val,87,[VOCAB]
35446,what time does the tv say it is?,2fda7dceebdfd8f2,"[Computer monitor, Television]",https://farm3.staticflickr.com/4022/4440187937...,https://c5.staticflickr.com/5/4022/4440187937_...,1024,767,"[9:25 am, wed 3/17, 9:25, 9:25 am, 9:25am, 9:2...","[what, time, does, the, tv, say, it, is]",val,9:25,[OCR]


In [24]:
metrics = evaluate(list(combo_df['answers']), list(combo_df['answer'])) #labels, preds

In [25]:
accuracies, avg_acc = metrics

In [26]:
#ACCURACY
avg_acc

0.37406666666666627

In [27]:
len(accuracies)

5000

In [28]:
combo_df['EVAL'] = accuracies

## ERROR ANALYSIS

I'm setting:

wrong: <= 0.35

right: > 0.35

In [29]:
right = combo_df[combo_df['EVAL'] > 0.35]
wrong = combo_df[combo_df['EVAL'] <= 0.35]

In [30]:
#how many different answers
len(Counter(list(combo_df['answers'])[10]))

7

In [31]:
#how many different answers!
Counter(len(Counter(item)) for item in right['answers']).most_common(n=10)

[(2, 514),
 (1, 503),
 (3, 389),
 (4, 234),
 (5, 115),
 (6, 51),
 (7, 18),
 (8, 14),
 (9, 1)]

In [32]:
#how many different answers!
Counter(len(Counter(item)) for item in wrong['answers']).most_common(n=10)

[(2, 703),
 (3, 663),
 (4, 519),
 (1, 450),
 (5, 346),
 (6, 198),
 (7, 135),
 (8, 90),
 (9, 37),
 (10, 20)]

In [33]:
Counter([source for pred_sources in wrong['pred_source'] for source in pred_sources])

Counter({'OCR': 2559, 'VOCAB': 2219})

In [34]:
Counter([source for pred_sources in right['pred_source'] for source in pred_sources])

Counter({'OCR': 1359, 'VOCAB': 1023})

In [35]:
Counter([tok for q_tokens in wrong['question_tokens'] for tok in q_tokens]).most_common(30)

[('the', 3154),
 ('what', 2626),
 ('is', 2211),
 ('of', 1019),
 ('on', 868),
 ('this', 671),
 ('in', 345),
 ('name', 318),
 ('does', 289),
 ('brand', 267),
 ('number', 256),
 ('are', 239),
 ("'", 182),
 ('s', 176),
 ('to', 173),
 ('book', 167),
 ('say', 166),
 ('who', 162),
 ('kind', 140),
 ('right', 136),
 ('how', 134),
 ('it', 125),
 ('written', 121),
 ('left', 120),
 ('sign', 112),
 ('time', 112),
 ('word', 110),
 ('for', 109),
 ('top', 102),
 ('title', 99)]

In [36]:
Counter([tok for q_tokens in right['question_tokens'] for tok in q_tokens]).most_common(30)

[('the', 1561),
 ('what', 1337),
 ('is', 1321),
 ('this', 506),
 ('of', 495),
 ('on', 469),
 ('number', 228),
 ('in', 197),
 ('brand', 194),
 ('does', 167),
 ('name', 124),
 ('are', 107),
 ('a', 94),
 ('sign', 88),
 ('say', 87),
 ('who', 84),
 ('right', 74),
 ('to', 74),
 ("'", 72),
 ('kind', 70),
 ('s', 69),
 ('player', 68),
 ('top', 65),
 ('year', 65),
 ('it', 64),
 ('left', 64),
 ('that', 63),
 ('phone', 62),
 ('book', 61),
 ('was', 60)]

In [37]:
Counter([subitem for item in wrong['answers'] for subitem in item]).most_common(30)

[('unanswerable', 680),
 ('answering does not require reading text in the image', 279),
 ('no', 207),
 ('1', 172),
 ('3', 168),
 ('5', 144),
 ('2', 134),
 ('yes', 120),
 ('4', 118),
 ('9', 110),
 ('6', 108),
 ('8', 107),
 ('a', 86),
 ('11', 80),
 ('lg', 76),
 ('10', 73),
 ('12', 70),
 ('t', 69),
 ('7', 67),
 ('red', 61),
 ('rolex', 60),
 ('beer', 59),
 ('s', 59),
 ('google', 58),
 ('apple', 55),
 ('not a question', 54),
 ('coca cola', 54),
 ('20', 53),
 ('p', 53),
 ('0', 50)]

In [38]:
Counter([subitem for item in right['answers'] for subitem in item]).most_common(30)

[('yes', 1884),
 ('unanswerable', 358),
 ('answering does not require reading text in the image', 353),
 ('no', 218),
 ('stop', 213),
 ('lg', 124),
 ('htc', 118),
 ('3', 105),
 ('5', 90),
 ('samsung', 85),
 ('2013', 84),
 ('2011', 81),
 ('16', 68),
 ('2010', 68),
 ('black', 63),
 ('2', 62),
 ('sony', 60),
 ('9', 59),
 ('30', 57),
 ('35', 57),
 ('1', 56),
 ('nokia', 56),
 ('virginia', 56),
 ('33', 54),
 ('22', 53),
 ('10', 52),
 ('19', 51),
 ('8', 50),
 ('21', 49),
 ('2009', 48)]

In [39]:
Counter([item for item in wrong['answer']]).most_common(30)

[('unanswerable', 230),
 ('answering does not require reading text in the image', 76),
 ('yes', 49),
 ('2', 38),
 ('1', 24),
 ('8', 23),
 ('5', 23),
 ('12', 19),
 ('20', 16),
 ('0', 15),
 ('6', 14),
 ('no text in image', 14),
 ('m', 11),
 ('apple', 10),
 ('a', 10),
 ('3', 9),
 ('100', 9),
 ('4', 8),
 ('n', 8),
 ('10', 8),
 ('no', 7),
 ('7:20', 7),
 ('beer', 6),
 ('18', 6),
 ('sony', 6),
 ('15', 6),
 ('l', 6),
 ('stop', 6),
 ('chocolate', 6),
 ('lg', 5)]

In [40]:
Counter([item for item in right['answer']]).most_common(30)

[('yes', 241),
 ('answering does not require reading text in the image', 41),
 ('unanswerable', 40),
 ('stop', 26),
 ('lg', 15),
 ('no', 14),
 ('htc', 12),
 ('5', 9),
 ('2013', 9),
 ('3', 9),
 ('samsung', 9),
 ('2011', 8),
 ('coca cola', 7),
 ('16', 7),
 ('2010', 7),
 ('22', 6),
 ('sony', 6),
 ('30', 6),
 ('nokia', 6),
 ('virginia', 6),
 ('9', 6),
 ('35', 6),
 ('21', 6),
 ('33', 6),
 ('black', 5),
 ('b', 5),
 ('2009', 5),
 ('california', 5),
 ('1', 5),
 ('2', 5)]

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
vectorizer = TfidfVectorizer()

In [43]:
corpus = [" ".join(item).strip() for item in right['question_tokens']]

In [44]:
X = vectorizer.fit_transform(corpus)

In [45]:
indices = np.argsort(vectorizer.idf_)[::-1]
features = vectorizer.get_feature_names()
top_n = 20
top_features = [features[i] for i in indices[:top_n]]
top_features

['zone',
 'held',
 'hats',
 'having',
 'head',
 'headed',
 'heading',
 'heater',
 'helix',
 'grape',
 'highway',
 'hillary',
 'him',
 'hispanic',
 'hockey',
 'hocky',
 'hates',
 'happy',
 'happening',
 'handwritten']

In [46]:
#IDEAS
#tfidf on the question tokens or look at spacy POS tags
#cluster visual resnet embeddings

In [51]:
dataset = wrong['image_classes'].to_list()

In [52]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
binary_mat = pd.DataFrame(te_ary, columns=te.columns_)
binary_mat

,Aircraft,Airplane,Alarm clock,Ambulance,Animal,Apple,Auto part,Backpack,Baked goods,Ball,Balloon,Band-aid,Banjo,Barrel,Baseball bat,Baseball glove,Bat,Bathroom accessory,Bathroom cabinet,Beaker,Bear,Bed,Beer,Belt,Bench,Bicycle,Bicycle helmet,Bicycle wheel,Bidet,Billboard,Bird,Blender,Blue jay,Boat,Bomb,Book,Bookcase,Bottle,Bottle opener,Bowl,...,Tie,Tin can,Tire,Toaster,Toilet,Toilet paper,Tool,Towel,Tower,Toy,Traffic light,Traffic sign,Train,Tree,Trousers,Truck,Umbrella,Van,Vase,Vegetable,Vehicle,Vehicle registration plate,Volleyball,Waffle iron,Wall clock,Wardrobe,Washing machine,Waste container,Watch,Watercraft,Wheel,Whisk,Whiteboard,Window,Wine,Wine rack,Wok,Woman,Worm,Wrench
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3156,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3157,False,False,False,False,False,False,False,False,False,False,False

In [53]:
apriori(binary_mat, min_support=0.1, use_colnames=True)

,support,itemsets
0,0.149953,(Book)
1,0.141095,(Bottle)
2,0.178741,(Clothing)
3,0.134135,(Drink)
4,0.128440,(Man)
5,0.276811,(Person)
6,0.108510,(Poster)
7,0.122430,(Sports uniform)
8,0.114204,"(Drink, Bottle)"
9,0.147422,"(Person, Clothing)"


In [54]:
dataset = right['image_classes'].to_list()

In [55]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
binary_mat = pd.DataFrame(te_ary, columns=te.columns_)
binary_mat

,Aircraft,Airplane,Alarm clock,Ambulance,Apple,Auto part,Backpack,Baked goods,Ball,Balloon,Band-aid,Banjo,Barge,Barrel,Baseball bat,Baseball glove,Bat,Beaker,Bear,Bed,Beer,Belt,Bench,Bicycle,Bicycle helmet,Bicycle wheel,Bidet,Billboard,Billiard table,Bird,Blender,Boat,Bomb,Book,Bookcase,Boot,Bottle,Bottle opener,Bowl,Box,...,Tent,Tie,Tin can,Tire,Toaster,Toilet,Toilet paper,Tool,Towel,Tower,Toy,Traffic light,Traffic sign,Train,Tree,Trousers,Truck,Trumpet,Van,Vase,Vegetable,Vehicle,Vehicle registration plate,Volleyball,Wall clock,Washing machine,Waste container,Watch,Watercraft,Weapon,Wheel,Whisk,Whiteboard,Window,Wine,Wine rack,Wok,Woman,Wood-burning stove,Worm
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1835,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [56]:
apriori(binary_mat, min_support=0.1, use_colnames=True)

,support,itemsets
0,0.137575,(Bottle)
1,0.209897,(Clothing)
2,0.127243,(Drink)
3,0.159869,(Man)
4,0.301251,(Person)
5,0.117455,(Sports equipment)
6,0.172376,(Sports uniform)
7,0.101686,(Vehicle)
8,0.115280,"(Drink, Bottle)"
9,0.130506,"(Clothing, Man)"
